# Project Introduction:
Primer laboratorio de aprendizaje automático, desarrollado por Sergio Barragán Blanco (100472343) y Eduardo Alarcón Navarro (100472175). 
Grupo 17.


# 1. EDA
Existen 23 carácterísticas que definen cada momento, de las cuales ninguna es categórica, todas son numéricas (con la energía suman 24). No existen valores faltantes, pero si los hubiera, los rellenaríamos con la media del valor superior e inferior antes de randomizar el dataset.

No existen tampoco columnas constantes, que se eliminarían. 

Con todo esto, podemos observar que es un problema de regresión.

La variable que estamos intentando predecir es la "energía" que es el valor de la enrgía generada 24 horas después. 

## Correlaciones entre parámetros:
Al principio, parece que las columnas lai_lv.13 y lai_hv.13 tienen una correlación, pero según avanza el tiempo, desaparece.

## Escala de los datos
Entre las diferentes columnas de datos, tenemos valores y magnitudes muy dispares.

In [22]:
import pandas as pd
import numpy as np
# Load the data
data = pd.read_csv('wind_ava.csv.gz', compression='gzip')
# FIlter the data to only include the columns that end in 13
import time
a = time.time()
data = data.filter(regex='13$|energy')
#print(data)
print(time.time()-a)



0.006005287170410156


In [23]:
from sklearn.model_selection import train_test_split

y = data['energy']
x = data.drop(columns=['energy'])


X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=7543)


In [15]:
# Preprocessing

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import Normalizer
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import KFold, RandomizedSearchCV
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier  
from sklearn.pipeline import Pipeline

scalers = [StandardScaler(), MinMaxScaler(), RobustScaler(), Normalizer()]

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), x.columns)
    ])


In [9]:
# Optimizing hyperparameters
"""

param_grid = {
    'criterion': ['friedman_mse', 'absolute_error', 'poisson', 'squared_error'],  # Adjust criterion for regression
    'max_depth': [None, 7, 10, 12],
    'min_samples_split': [10, 15, 20, 30],
    'min_samples_leaf': [4, 5, 6]
}

cv = KFold(n_splits=10)
model = RandomizedSearchCV(DecisionTreeRegressor(random_state=7543),  # Use DecisionTreeRegressor
                     param_grid, cv=cv, n_jobs=-1, verbose=1, scoring='neg_mean_absolute_percentage_error', n_iter=50)  # Adjust scoring metric

pipe = Pipeline([
    ('preprocessor', preprocessor),
    ('model', model)
])

pipe.fit(X_train, y_train)
y_pred = pipe.predict(X_test)

print(f"Best params: {model.best_params_}")
"""


'\nfrom sklearn.model_selection import KFold, RandomizedSearchCV\nfrom sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier  \nfrom sklearn.pipeline import Pipeline\n\n\nparam_grid = {\n    \'criterion\': [\'friedman_mse\', \'absolute_error\', \'poisson\', \'squared_error\'],  # Adjust criterion for regression\n    \'max_depth\': [None, 7, 10, 12],\n    \'min_samples_split\': [10, 15, 20, 30],\n    \'min_samples_leaf\': [4, 5, 6]\n}\n\ncv = KFold(n_splits=10)\nmodel = RandomizedSearchCV(DecisionTreeRegressor(random_state=7543),  # Use DecisionTreeRegressor\n                     param_grid, cv=cv, n_jobs=-1, verbose=1, scoring=\'neg_mean_absolute_percentage_error\', n_iter=50)  # Adjust scoring metric\n\npipe = Pipeline([\n    (\'preprocessor\', preprocessor),\n    (\'model\', model)\n])\n\npipe.fit(X_train, y_train)\ny_pred = pipe.predict(X_test)\n\nprint(f"Best params: {model.best_params_}")\n'

In [25]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Crear el modelo de regresión lineal
model = LinearRegression()

# Entrenar el modelo con los datos de entrenamiento
model.fit(X_train, y_train)

# Realizar predicciones en el conjunto de prueba
y_pred = model.predict(X_test)

# Calcular el error cuadrático medio (MSE) en el conjunto de prueba
mse = mean_squared_error(y_test, y_pred)

print("MSE:", mse)



MSE: 313274.75857536943


In [20]:
# Vamos a probar con KNN Regresor
from sklearn import neighbors
from sklearn import metrics
regr = neighbors.KNeighborsRegressor()

# We train it
regr.fit(X_train, y_train)

# We obtain predictions on the test set
y_test_pred = regr.predict(X_test)

# We compute accuracy
rmse_knn = np.sqrt(metrics.mean_squared_error(y_test, y_test_pred))
print(f"RMSE of the model: {rmse_knn}")

RMSE of the model: 621.2880027885399
